In [2]:
import requests
import websocket
import json
from websocket import create_connection
import time
import pandas as pd
import numpy as np
from scipy.stats import norm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
from IPython.display import clear_output
import random
from scipy.optimize import fsolve

In [918]:
# Volatility index parameters such as vol, displayed decimals
assets=pd.read_csv("assets.csv")
#API URL
master_URL = "wss://qa69.deriv.dev/websockets/v3?app_id=16929&l=EN&brand=deriv"
#api token
api_token = "yO3kHGwRauw84xX"

In [920]:
def barrier_check (symbol, duration, duration_unit, contract_type): 
    if duration_unit == "d":
        barrier = "1000"        
    else:        
        barrier = "+0"        
    return_json = []
    def on_open(ws):
        msg = {
            "proposal": 1,
            "amount": 0.01,
            "barrier": barrier,
            "basis": "stake",
            "contract_type": contract_type,
            "currency": "BTC",
            "duration": duration,
            "duration_unit": duration_unit,
            "symbol": symbol
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message): 
        amount = 0.01    
        message = json.loads(message)        
        if 'error' in message.keys():            
            barrier = message['error']['details']['barrier_choices'][int(len(message['error']['details']['barrier_choices'])/2)]                
            if message['error']['details']['field'] == 'amount':               
                amount = float(message['error']['details']['max_stake'])
            msg = {
                "proposal": 1,
                "amount": amount,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "BTC",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            ws.send(json.dumps(msg))              
        else:
            return_json.append(message)
            ws.close()            
    def on_close(ws, close_status_code, close_msg):
        print()              
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    barriers = return_json[-1]['proposal']['barrier_choices']
    return return_json[-1]

In [921]:
def calc_barriers (pd_series): #(spot, symbol, duration, duration_unit):    
    spot = float(pd_series.Spot)    
    symbol = pd_series.Asset    
    duration = pd_series.Start_expiry    
    duration_unit = pd_series.Duration[-1]    
    digits = int( assets[assets['Asset']== symbol].digits )    
    sigma = float( assets[assets['Asset']==symbol].IV )     
    n_digits = round( - np.log10(spot * sigma * np.sqrt(60/365/86400)) , 0) - 1    
    spot = round(spot/10**-n_digits)*10**-n_digits     
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400           
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400        
    if duration_unit != 'd':        
        deltas = np.arange(1,10,2)/10     
        real_strikes = spot * np.exp( (sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(deltas)) )    
        displayed_barriers = np.where( deltas > 0.5, np.round((real_strikes - spot)*10**digits)/10**digits , np.round((real_strikes - spot)*10**digits)/10**digits )        
    else:        
        temp_barriers = []        
        displayed_barriers = []
        central_strike = round( spot / (10 ** -n_digits) ) * (10 ** - n_digits )
        max_strike =  np.floor( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.1)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )
        min_strike =  np.ceil( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.9)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )
        strike_step_1 = round( (central_strike - min_strike) / 5 / 10 ) * 10
        strike_step_2 = round( (max_strike - central_strike) / 5 / 10 ) * 10 
        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1*i)
        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2*i)
        temp_barriers.append(max_strike)
        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)
    return displayed_barriers        

In [1030]:
def calc_ppp_ask (pd_series, t_step, s_step):
    symbol = pd_series.Asset    
    duration = pd_series.Start_expiry    
    duration_unit = pd_series.Duration[-1]   
    stake = pd_series.Stake
    vol_charge = 1.025 
    t = ( int(duration[1]) - int(duration[0]) + t_step ) / 365 / 86400      
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        print('rounded')
        t -= 2 / 365 / 86400
    r = pd_series.Discount_rate    
    contract_type = pd_series.Contract_type
    number_of_contracts = pd_series.API_PPP       
    if s_step == 0:        
        spot = float(pd_series.Spot)
        strike = pd_series.Barrier   
    else:        
        spot = float(tick_history(int(duration[0]), symbol, s_step))       
        if duration_unit != 'd':           
            strike = spot + float(pd_series.Displayed_barrier)            
        else:            
            strike = pd_series.Barrier        
    sigma = float( assets[assets['Asset']==symbol].IV )    
    if duration_unit != 'd':        
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )        
    else:        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))    
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2        
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2        
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))    
    print(symbol,'s', spot, 'k',strike, r, t, vol,  phi,  'p',number_of_contracts, stake, 'd', (norm.cdf(phi*d1)))    
    return stake / ( bs_price + delta_charge + bs_markup )

In [923]:
def tick_history (tick,symbol,t):   
    if 'R' in symbol:
        r = 2
    else:
        r = 1        
    temp_json = []    
    def on_open(ws):
        omsg = {
          "ticks_history": symbol,
          "adjust_start_time": 1,
          "count": 5,
          "end": tick+t*r,
          "start": 1,
          "style": "ticks"
        }
        ws.send(json.dumps(omsg))        
    def on_message(ws, message):
        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            ws.send(json.dumps(msg))            
        else:        
            temp_json.append(message)
            ws.close()    
    def on_close(ws, close_status_code, close_msg):
        print()        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    return temp_json[-1]['history']['prices'][-1]    

In [924]:
def get_proposal (contract_id):     
    contract_id = int(contract_id)    
    return_json = []    
    def on_open(ws):        
        msg = {
            "authorize": api_token,
            "req_id": 1
        }        
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        print(message)
        if 'authorize' in message:            
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_id
            }            
            ws.send(json.dumps(msg))           
        else:                   
            return_json.append(message)            
            ws.close()
    def on_close(ws, close_status_code, close_msg):
        print()
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    return return_json[-1]

In [1152]:
def get_barriers (symbol, duration, duration_unit, contract_type):     
    barrier = 0    
    if duration_unit == "d":
        barrier = "1000"
    else:
        barrier = "+0"        
    return_json = []
    def on_open(ws):        
        msg = {
            "proposal": 1,
            "amount": 0.01,
            "barrier": barrier,
            "basis": "stake",
            "contract_type": contract_type,
            "currency": "BTC",
            "duration": duration,
            "duration_unit": duration_unit,
            "symbol": symbol
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        if duration_unit == "d":
            barrier = "1000"
        else:
            barrier = "+0"        
        print('\n',message)
        amount = 0.01
        message = json.loads(message)
        if 'error' in message.keys():            
            if message['error']['details']['field'] == 'barrier':                
                barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]                
            elif message['error']['details']['field'] == 'amount':
                amount = round(float(message['error']['details']['min_stake']) + 0.0001,9)
                barrier = message['echo_req']['barrier']
            msg = {
                "proposal": 1,
                "amount": amount,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "BTC",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print(msg)
            ws.send(json.dumps(msg))              
        else:
            return_json.append(message)
            ws.close()            
    def on_close(ws, close_status_code, close_msg):
        print()             
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    barriers = return_json[-1]['proposal']['barrier_choices']    
    if 'qa' in master_URL:       
        return barriers, return_json[-1]['proposal']['pricing_args']['discount_rate']    
    else:        
        return barriers, 0

In [1178]:
def get_contract (symbol, duration, duration_unit, contract_type): 
    return_json = []
    return_barrier = []    
    return_discount_rate = []
    contract_ID = []    
    def on_open(ws):        
        msg = {
            "authorize": api_token,
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        print('\n',message)
        barrier = 0        
        amount = 0.03        
        message = json.loads(message)        
        if 'authorize' in message:
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "BTC",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))    
        elif 'error' in message.keys():            
            if 'code' in message['error'].keys():
                amount = float(message['error']['message'].split()[-1][:-1])                 
                if 'ax' in message['error']['message']:
                    amount -= 0.0001
                else: 
                    amount += 0.0001  
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "BTC",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))
        elif 'buy' in message:
            contract_ID.append(message['buy']['contract_id'])
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_ID[-1]
            }
            ws.send(json.dumps(msg))           
        else:
            return_json.append(message)           
            ws.close()                                          
    def on_close(ws, close_status_code, close_msg):
        print()
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    return return_json[-1], return_discount_rate[-1], return_barrier[-1]

In [1157]:
def calc_ppp_bid (proposal, t_step, s_step):
    symbol = proposal['proposal_open_contract']['underlying']   
    stake = float(proposal['proposal_open_contract']['bid_price'])
    vol_charge = 0.975 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['current_spot_time'] + t_step ) / 365 / 86400          
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400        
    if t > 1 / 365:        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    else:        
        r = 0                
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])        
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['current_spot'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(int(proposal['proposal_open_contract']['current_spot_time']), symbol, s_step))        
        if t > 1 / 365:        
            strike = float(proposal['proposal_open_contract']['barrier'])
        else:
            strike = spot + float(proposal['proposal_open_contract']['barrier']) - float(proposal['proposal_open_contract']['current_spot'])    
    sigma = float( assets[assets['Asset']==symbol].IV )       
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1       
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2   
    print(spot,strike,r, t, phi,  number_of_contracts, stake)
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))
    return ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup) * number_of_contracts   

In [1093]:
def sell_contract(contract_id):    
    contract_id = int(contract_id)   
    temp_json = []    
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))        
    def on_message(ws, message):
        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:            
            msg = {
              "sell": contract_id,
              "price": 0
            }
            ws.send(json.dumps(msg))            
        else:        
            temp_json.append(float(message['sell']['sold_for']))
            ws.close()    
    def on_close(ws, close_status_code, close_msg):
        print()        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    return temp_json[-1]

In [1135]:
def calc_ppp_sold (proposal, t_step, s_step):
    symbol = proposal['proposal_open_contract']['underlying']    
    stake = float(proposal['proposal_open_contract']['sell_price'])
    vol_charge = 0.975
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['exit_tick_time'] + t_step) / 365 / 86400  
    if t > 1 / 365:        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    else:       
        r = 0                
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )        
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])        
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))        
        strike = float(proposal['proposal_open_contract']['barrier'])    
    sigma = float( assets[assets['Asset']==symbol].IV )
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
    elif  contract_type == "VANILLALONGPUT":       
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2           
    print('s',spot,'k',strike,'r',r,'t', t, phi,'n',  number_of_contracts,'s', stake)
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))    
    price = ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup)
    print(price)
    return price * number_of_contracts  

In [599]:
def calc_ppp_exp (proposal, t_step, s_step):    
    stake = float(proposal['proposal_open_contract']['sell_price'])    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])        
    if s_step == 0:       
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))        
        strike = float(proposal['proposal_open_contract']['barrier'])    
    if contract_type == "VANILLALONGCALL":
        phi = 1
    elif  contract_type == "VANILLALONGPUT":
        phi = -1               
    print(spot,strike, phi,  number_of_contracts, stake)
    return max( (spot - strike) * phi * number_of_contracts, 0)

In [ ]:
# barrier_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Duration', 'Start_expiry', 'API_barrier_choice', 'Calc_barrier_choices', 'json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait = True)    
    print(len(barrier_df))
    if len(barrier_df) > 30 - 1:
        break                
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':        
        max_duration = 1440
    elif duration_unit == 'h':        
        max_duration = 24
    elif duration_unit == 'd':        
        max_duration = 365
    i = np.random.randint(5)    
    if i < 2:        
        duration = max_duration**i        
    else:        
        duration = np.random.randint(1,max_duration)        
    price_proposal = barrier_check(asset, duration, duration_unit, contract_type)   
    barrier_df.loc[len(barrier_df)] = [ price_proposal['echo_req']['symbol'], price_proposal['echo_req']['contract_type'], price_proposal['proposal']['spot'], \
                                        price_proposal['proposal']['spot_time'], str(price_proposal['echo_req']['duration']) + price_proposal['echo_req']['duration_unit'], \
                                        [ price_proposal['proposal']['date_start'], price_proposal['proposal']['date_expiry'] ], \
                                        [float(x) for x in price_proposal['proposal']['barrier_choices']], 0, price_proposal ]   
clear_output(wait = True)
barrier_df

In [1134]:
Calc_barrier_choices = [] 
for i in barrier_df.index:    
    Calc_barrier_choices.append( calc_barriers(barrier_df.iloc[i]) )
barrier_df.Calc_barrier_choices = Calc_barrier_choices
print(len(barrier_df), 'Test cases','\nMismatches =',\
      len(barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) != 0]),\
      '\nMismatches listed below :')
barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) != 0]

20 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json


In [ ]:
# PPP_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Calc_PPP','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait=True)
    print(len(PPP_df))    
    if len(PPP_df)>250-1:
        break        
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)   
    if i<2:        
        duration = max_duration**i       
    else:        
        duration = np.random.randint(1, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)   
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]   
    PPP_df.loc[len(PPP_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, proposal]
clear_output(wait=True)
PPP_df

In [1088]:
Calc_ppp = [] 
for i in PPP_df.index:    
    Calc_ppp.append( calc_ppp_ask(PPP_df.iloc[i], 0, 0) )
PPP_df.Calc_PPP = Calc_ppp
clear_output(wait = True)
print(len(PPP_df), 'Test cases','\nMismatches =',\
      len(PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,10) != 0]),\
      '\nMismatches listed below :')
df = PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,10) != 0]
df

249 Test cases 
Mismatches = 3 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
30,R_25,VANILLALONGPUT,1912.7270,1683698100,0.01,1m,-0.830,1911.8970,"[1683698101, 1683698161]",639,0,0.296808,0.296808,"{'echo_req': {'contract_id': 639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200021', 'barrier': '1911.897', 'barrier_count': 1, 'bid_price': '0.00776730', 'buy_price': '0.01000000', 'contract_id': '639', 'contract_type': 'VANILL..."
54,1HZ10V,VANILLALONGCALL,9182.8700,1683698703,0.01,283d,9000.00,9000.0000,"[1683698703, 1708214399]",1139,0,0.000023,0.000023,"{'echo_req': {'contract_id': 1139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200021', 'barrier': '9000.00', 'barrier_count': 1, 'bid_price': 0.00963595, 'buy_price': '0.01000000', 'contract_id': '1139', 'contract_type': 'VANILLA..."
145,R_75,VANILLALONGCALL,374870.3462,1683700432,0.01,8h,+95.0460,374965.3922,"[1683700432, 1683729232]",3019,0,0.000003,0.000003,"{'echo_req': {'contract_id': 3019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200041', 'barrier': '374965.3922', 'barrier_count': 1, 'bid_price': '0.00950390', 'buy_price': '0.01000000', 'contract_id': '3019', 'contract_type': 'V..."


In [1278]:
# Bid_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:    
    print(len(Bid_df))    
    if len(Bid_df)>100-1:
        break            
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    duration = np.random.randint(max_duration/2, max_duration)    
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Bid_df.loc[len(Bid_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
    clear_output(wait=True)
Bid_df

,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,R_50,VANILLALONGPUT,150.8079,1683772894,0.01,1140m,+0.0410,150.8489,"[1683772895, 1683841295]",7699,0,0.006865,1683773742,0.008983,0.008983,"{'echo_req': {'contract_id': 7699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '150.8489', 'barrier_count': 1, 'bid_price': 0.00898293, 'buy_price': '0.01000000', 'contract_id': '7699', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '150.8489', 'barrier_count': 1, 'bid_price': 0.00951914, 'buy_price': '0.01000000', 'contract_id': '7699', 'contract_type': 'VANILL..."
1,1HZ25V,VANILLALONGPUT,446821.6200,1683772900,0.01,1143m,-2694.30,444127.3200,"[1683772900, 1683841480]",7719,0,0.000010,1683773746,0.009982,0.009982,"{'echo_req': {'contract_id': 7719, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '444127.32', 'barrier_count': 1, 'bid_price': 0.00998204, 'buy_price': '0.01000000', 'contract_id': '7719', 'contract_type': 'VANIL...","{'echo_req': {'contract_id': 7719, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '444127.32', 'barrier_count': 1, 'bid_price': 0.00913165, 'buy_price': '0.01000000', 'contract_id': '7719', 'contract_type': 'VANIL..."
2,R_25,VANILLALONGPUT,1915.6130,1683772904,0.01,246d,2550.000,2550.0000,"[1683772906, 1705103999]",7739,0,0.000015,1683773748,0.009957,0.009957,"{'echo_req': {'contract_id': 7739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '2550.000', 'barrier_count': 1, 'bid_price': 0.00995741, 'buy_price': '0.01000000', 'contract_id': '7739', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '2550.000', 'barrier_count': 1, 'bid_price': 0.00994373, 'buy_price': '0.01000000', 'contract_id': '7739', 'contract_type': 'VANILL..."
3,R_100,VANILLALONGCALL,4923.3700,1683772910,0.01,912m,+273.20,5196.5700,"[1683772911, 1683827631]",7759,0,0.000944,1683773750,0.007051,0.007052,"{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00705128, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00828796, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA..."
4,R_25,VANILLALONGPUT,1915.8560,1683772916,0.01,227d,1600.000,1600.0000,"[1683772916, 1703462399]",7779,0,0.000277,1683773752,0.008787,0.008787,"{'echo_req': {'contract_id': 7779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '1600.000', 'barrier_count': 1, 'bid_price': 0.00878689, 'buy_price': '0.01000000', 'contract_id': '7779', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '1600.000', 'barrier_count': 1, 'bid_price': '0.00874770', 'buy_price': '0.01000000', 'contract_id': '7779', 'contract_type': 'VANI..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1HZ25V,VANILLALONGCALL,4

In [1158]:
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in Bid_df.index:   
    if i >= len(bid_time):       
        clear_output(wait = True)
        proposal = get_proposal(Bid_df.loc[i].ID)
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, 0, 0) )        
        bid_prop.append(proposal)    
Bid_df.Bid_time = bid_time
Bid_df.API_bid = api_bid
Bid_df.Calc_bid = calc_bid
Bid_df.bid_prop = bid_prop
clear_output(wait = True)
print(len(Bid_df), 'Test cases','\nMismatches =',\
      len(Bid_df[round(Bid_df.API_bid,8) - round(Bid_df.Calc_bid,8) != 0]),\
      '\nMismatches listed below :')
df = Bid_df[round(Bid_df.API_bid,8) - round(Bid_df.Calc_bid,8) != 0]
Bid_df[round(Bid_df.API_bid,8) - round(Bid_df.Calc_bid,8) != 0]

80 Test cases 
Mismatches = 30 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,R_50,VANILLALONGPUT,150.8079,1683772894,0.0100,1140m,+0.0410,150.8489,"[1683772895, 1683841295]",7699,0,6.864602e-03,1683773742,0.008983,0.008983,"{'echo_req': {'contract_id': 7699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '150.8489', 'barrier_count': 1, 'bid_price': 0.00898293, 'buy_price': '0.01000000', 'contract_id': '7699', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '150.8489', 'barrier_count': 1, 'bid_price': 0.00951914, 'buy_price': '0.01000000', 'contract_id': '7699', 'contract_type': 'VANILL..."
3,R_100,VANILLALONGCALL,4923.3700,1683772910,0.0100,912m,+273.20,5196.5700,"[1683772911, 1683827631]",7759,0,9.444240e-04,1683773750,0.007051,0.007052,"{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00705128, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00828796, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA..."
5,R_10,VANILLALONGCALL,6690.7690,1683772920,0.0100,22h,+43.190,6733.9590,"[1683772921, 1683852121]",7799,0,5.765034e-03,1683773754,0.010436,0.010437,"{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '6733.959', 'barrier_count': 1, 'bid_price': 0.01043643, 'buy_price': '0.01000000', 'contract_id': '7799', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '6733.959', 'barrier_count': 1, 'bid_price': 0.00838459, 'buy_price': '0.01000000', 'contract_id': '7799', 'contract_type': 'VANILL..."
8,1HZ10V,VANILLALONGPUT,9144.8800,1683772938,0.0100,22h,+24.20,9169.0800,"[1683772938, 1683852138]",7859,0,3.003451e-04,1683773762,0.008922,0.008922,"{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9169.08', 'barrier_count': 1, 'bid_price': 0.00892173, 'buy_price': '0.01000000', 'contract_id': '7859', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9169.08', 'barrier_count': 1, 'bid_price': 0.00976079, 'buy_price': '0.01000000', 'contract_id': '7859', 'contract_type': 'VANILLA..."
9,R_100,VANILLALONGPUT,4917.9300,1683772942,0.0100,1101m,-111.20,4806.7300,"[1683772943, 1683839003]",7879,0,2.174208e-04,1683773764,0.009869,0.009869,"{'echo_req': {'contract_id': 7879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '4806.73', 'barrier_count': 1, 'bid_price': 0.00986852, 'buy_price': '0.01000000', 'contract_id': '7879', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '4806.73', 'barrier_count': 1, 'bid_price': 0.00914835, 'buy_price': '0.01000000', 'contract_id': '7879', 'contract_type': 'VANILLA..."
10,R_100,VANILLALONGPUT,4904.5400,1683773054,0.0100,18h,-271.80,4632.7400,"[1

In [1290]:
Bid_df.head(40)

,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,R_50,VANILLALONGPUT,150.8079,1683772894,0.010000,1140m,+0.0410,150.8489,"[1683772895, 1683841295]",7699,0,6.864602e-03,1683773742,0.008983,0.008983,"{'echo_req': {'contract_id': 7699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '150.8489', 'barrier_count': 1, 'bid_price': 0.00898293, 'buy_price': '0.01000000', 'contract_id': '7699', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '150.8489', 'barrier_count': 1, 'bid_price': 0.00951914, 'buy_price': '0.01000000', 'contract_id': '7699', 'contract_type': 'VANILL..."
1,1HZ25V,VANILLALONGPUT,446821.6200,1683772900,0.010000,1143m,-2694.30,444127.3200,"[1683772900, 1683841480]",7719,0,9.588800e-06,1683773746,0.009982,0.009982,"{'echo_req': {'contract_id': 7719, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '444127.32', 'barrier_count': 1, 'bid_price': 0.00998204, 'buy_price': '0.01000000', 'contract_id': '7719', 'contract_type': 'VANIL...","{'echo_req': {'contract_id': 7719, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '444127.32', 'barrier_count': 1, 'bid_price': 0.00913165, 'buy_price': '0.01000000', 'contract_id': '7719', 'contract_type': 'VANIL..."
2,R_25,VANILLALONGPUT,1915.6130,1683772904,0.010000,246d,2550.000,2550.0000,"[1683772906, 1705103999]",7739,0,1.531240e-05,1683773748,0.009957,0.009957,"{'echo_req': {'contract_id': 7739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '2550.000', 'barrier_count': 1, 'bid_price': 0.00995741, 'buy_price': '0.01000000', 'contract_id': '7739', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '2550.000', 'barrier_count': 1, 'bid_price': 0.00994373, 'buy_price': '0.01000000', 'contract_id': '7739', 'contract_type': 'VANILL..."
3,R_100,VANILLALONGCALL,4923.3700,1683772910,0.010000,912m,+273.20,5196.5700,"[1683772911, 1683827631]",7759,0,9.444240e-04,1683773750,0.007051,0.007052,"{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00705128, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00828796, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA..."
4,R_25,VANILLALONGPUT,1915.8560,1683772916,0.010000,227d,1600.000,1600.0000,"[1683772916, 1703462399]",7779,0,2.765372e-04,1683773752,0.008787,0.008787,"{'echo_req': {'contract_id': 7779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '1600.000', 'barrier_count': 1, 'bid_price': 0.00878689, 'buy_price': '0.01000000', 'contract_id': '7779', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '1600.000', 'barrier_count': 1, 'bid_price': '0.00874770', 'buy_price': '0.01000000', 'contract_id': '7779', 'contract_type': 'VANI..."
5,R_10,VANILLALONGCALL,6690.7690,1683772920,0.010000,22h,+4

In [1159]:
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in df.index:    
    if i >= len(bid_time):        
        clear_output(wait = True)
        proposal = df.loc[i].bid_prop
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, -1, 0) )
        bid_prop.append(proposal)    
df.Bid_time = bid_time
df.API_bid = api_bid
df.Calc_bid = calc_bid
df.bid_prop = bid_prop
clear_output(wait = True)
print(len(df), 'Test cases','\nMismatches =',\
      len(df[round(df.API_bid,8) - round(df.Calc_bid,8) != 0]),\
      '\nMismatches listed below :')
df2 = df[round(df.API_bid,8) - round(df.Calc_bid,8) != 0]
df2

30 Test cases 
Mismatches = 3 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
5,R_10,VANILLALONGCALL,6690.7690,1683772920,0.01,22h,+43.190,6733.9590,"[1683772921, 1683852121]",7799,0,0.005765,1683773754,0.010436,0.010437,"{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '6733.959', 'barrier_count': 1, 'bid_price': 0.01043643, 'buy_price': '0.01000000', 'contract_id': '7799', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '6733.959', 'barrier_count': 1, 'bid_price': 0.00838459, 'buy_price': '0.01000000', 'contract_id': '7799', 'contract_type': 'VANILL..."
20,R_50,VANILLALONGCALL,150.8176,1683773198,0.01,20h,-4.5110,146.3066,"[1683773198, 1683845198]",8479,0,0.002128,1683773790,0.009980,0.009980,"{'echo_req': {'contract_id': 8479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '146.3066', 'barrier_count': 1, 'bid_price': '0.00998020', 'buy_price': '0.01000000', 'contract_id': '8479', 'contract_type': 'VANI...","{'echo_req': {'contract_id': 8479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '146.3066', 'barrier_count': 1, 'bid_price': 0.00993285, 'buy_price': '0.01000000', 'contract_id': '8479', 'contract_type': 'VANILL..."
24,R_100,VANILLALONGCALL,4916.0300,1683773218,0.01,12h,+241.50,5157.5300,"[1683773219, 1683816419]",8559,0,0.001066,1683773800,0.007203,0.007204,"{'echo_req': {'contract_id': 8559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5157.53', 'barrier_count': 1, 'bid_price': 0.00720329, 'buy_price': '0.01000000', 'contract_id': '8559', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 8559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5157.53', 'barrier_count': 1, 'bid_price': 0.00828857, 'buy_price': '0.01000000', 'contract_id': '8559', 'contract_type': 'VANILLA..."


In [1136]:
Sold_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
                               'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:    
    print(len(Sold_df))   
    if len(Sold_df) > 30 - 1:
        break    
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365        
    duration = np.random.randint(max_duration/2, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Sold_df.loc[len(Sold_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
    clear_output(wait=True)
    
Sold_df

30


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ50V,VANILLALONGPUT,771378.8400,1683706745,0.010000,855m,+156.60,771535.4400,"[1683706745, 1683758045]",6479,0,1.552800e-06,0,0,0,0,"{'echo_req': {'contract_id': 6479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '771535.44', 'barrier_count': 1, 'bid_price': 0.00951846, 'buy_price': '0.01000000', 'contract_id': '6479', 'contract_type': 'VANIL..."
1,1HZ50V,VANILLALONGCALL,771289.7300,1683706750,0.010000,1406m,+257.50,771547.2300,"[1683706750, 1683791110]",6499,0,1.245500e-06,0,0,0,0,"{'echo_req': {'contract_id': 6499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '771547.23', 'barrier_count': 1, 'bid_price': 0.00950477, 'buy_price': '0.01000000', 'contract_id': '6499', 'contract_type': 'VANIL..."
2,1HZ75V,VANILLALONGPUT,7096.6400,1683706755,0.010000,23h,+149.90,7246.5400,"[1683706755, 1683789555]",6519,0,4.922620e-05,0,0,0,0,"{'echo_req': {'contract_id': 6519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '7246.54', 'barrier_count': 1, 'bid_price': 0.00976666, 'buy_price': '0.01000000', 'contract_id': '6519', 'contract_type': 'VANILLA..."
3,R_10,VANILLALONGPUT,6682.9730,1683706760,0.010000,1162m,+0.070,6683.0430,"[1683706760, 1683776480]",6539,0,7.761341e-04,0,0,0,0,"{'echo_req': {'contract_id': 6539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '6683.043', 'barrier_count': 1, 'bid_price': 0.00951352, 'buy_price': '0.01000000', 'contract_id': '6539', 'contract_type': 'VANILL..."
4,1HZ10V,VANILLALONGPUT,9174.0700,1683706765,0.010000,197d,8690.00,8690.0000,"[1683706765, 1700783999]",6559,0,1.073187e-04,0,0,0,0,"{'echo_req': {'contract_id': 6559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '8690.00', 'barrier_count': 1, 'bid_price': 0.00892857, 'buy_price': '0.01000000', 'contract_id': '6559', 'contract_type': 'VANILLA..."
5,1HZ50V,VANILLALONGPUT,771625.4300,1683706773,0.019114,242d,662000.00,662000.0000,"[1683706773, 1704671999]",6579,0,2.671000e-07,0,0,0,0,"{'echo_req': {'contract_id': 6579, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '662000.00', 'barrier_count': 1, 'bid_price': 0.01770107, 'buy_price': 0.01911431, 'contract_id': '6579', 'contract_type': 'VANILLA..."
6,R_100,VANILLALONGCALL,5117.8200,1683706778,0.010000,351d,24220.00,24220.0000,"[1683706778, 1714089599]",6599,0,3.704130e-05,0,0,0,0,"{'echo_req': {'contract_id': 6599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '24220.00', 'barrier_count': 1, 'bid_price': '0.00795790', 'buy_price': '0.01000000', 'contract_id': '6599', 'contract_type': 'VANI..."
7,R_75,VANILLALONGCALL,379639.1912,1683706786,0.015053,207d,920000.0000,920000.0000,"[1683706787, 1701647999]",6619,0,1.642200e-06,0,0,0,0,"{'echo_req': {'contract_id': 6619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '920000.0000', 'barrier_count': 1, 'bid_price': 0.01196011, 'buy_price': 0.01505289, 'contract_id': '6619', 'contract_type': 'VANIL..."
8,1HZ25V,VANILLALONGCALL,452951.8500,1683706792,0.010000,269d,484400.00,484400.0000,"[1683706792, 1707004799]",6639,0,3.668000e-07,0,0,0,0,"{'echo_req': {'contract_id': 6639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200141', 'barrier': '484400.00', 'barrier_count': 1, 'bid_price': 

In [1232]:
# calc_sold = []
# sold_time = []
# api_sold = []
# sold_prop = []
for i in Sold_df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)        
        sell_contract(Sold_df.loc[i].ID)        
        proposal = get_proposal(Sold_df.loc[i].ID)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )        
        sold_prop.append(proposal)    
Sold_df['Sold_time'] = sold_time
Sold_df['API_sold'] = api_sold
Sold_df['Calc_sold'] = calc_sold
Sold_df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[round(Sold_df.API_sold,8) - np.round(Sold_df.Calc_sold,8) != 0]),\
      '\nMismatches listed below :')
df = Sold_df[round(Sold_df.API_sold,8) - np.round(Sold_df.Calc_sold,8) != 0]
df

93 Test cases 
Mismatches = 26 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,...,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json,Sold_time,API_sold,Calc_sold,Sold_prop
3,R_100,VANILLALONGCALL,4923.3700,1683772910,0.010000,912m,+273.20,5.196570e+03,"[1683772911, 1683827631]",7759,...,9.444240e-04,1683773750,0.007051,0.007052,"{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00705128, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00828796, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA...",1683777506,0.004771,0.004771,"{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683772906, 'tick': '4921.97', 'tick_display_value': '4921.97'}, {'epoch': 1683772908, 'tick': ..."
5,R_10,VANILLALONGCALL,6690.7690,1683772920,0.010000,22h,+43.190,6.733959e+03,"[1683772921, 1683852121]",7799,...,5.765034e-03,1683773754,0.010436,0.010437,"{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '6733.959', 'barrier_count': 1, 'bid_price': 0.01043643, 'buy_price': '0.01000000', 'contract_id': '7799', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '6733.959', 'barrier_count': 1, 'bid_price': 0.00838459, 'buy_price': '0.01000000', 'contract_id': '7799', 'contract_type': 'VANILL...",1683777516,0.008699,0.008699,"{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683772916, 'tick': '6690.969', 'tick_display_value': '6690.969'}, {'epoch': 1683772918, 'tick'..."
7,1HZ10V,VANILLALONGPUT,9144.2600,1683772932,0.010000,755m,+44.60,9.188860e+03,"[1683772932, 1683818232]",7839,...,2.155534e-04,1683773760,0.008999,0.008999,"{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9188.86', 'barrier_count': 1, 'bid_price': 0.00899908, 'buy_price': '0.01000000', 'contract_id': '7839', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9188.86', 'barrier_count': 1, 'bid_price': 0.00994033, 'buy_price': '0.01000000', 'contract_id': '7839', 'contract_type': 'VANILLA...",1683777527,0.008753,0.008797,"{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683772930, 'tick': '9144.1', 'tick_display_value': '9144.10'}, {'epoch': 1683772931, 'tick': '..."
8,1HZ10V,VANILLALONGPUT,9144.8800,1683772938,0.010000,22h,+24.20,9.169080e+03,"[1683772938, 1683852138]",7859,...,3.003451e-04,1683773762,0.008922,0.008922,"{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9169.08', 'barrier_count': 1, 'bid_price': 0.00892173, 'buy_price': '0.01000000', 'contract_id': '7859', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'pro

In [1265]:
calc_sold = [] 
sold_time = []
api_sold = []
sold_prop = []
for i in df.index:   
    if i >= len(sold_time):        
        clear_output(wait = True)        
        proposal = (df.loc[i].Sold_prop)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, -1, 0) )        
        sold_prop.append(proposal)    
df['Sold_time'] = sold_time
df['API_sold'] = api_sold
df['Calc_sold'] = calc_sold
df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(df), 'Test cases','\nMismatches =',\
      len(df[round(df.API_sold,8) - np.round(df.Calc_sold,8) != 0]),\
      '\nMismatches listed below :')
df2 = df[round(df.API_sold,8) - np.round(df.Calc_sold,8) != 0]
df2

26 Test cases 
Mismatches = 13 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,...,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json,Sold_time,API_sold,Calc_sold,Sold_prop
7,1HZ10V,VANILLALONGPUT,9144.2600,1683772932,0.010000,755m,+44.60,9.188860e+03,"[1683772932, 1683818232]",7839,...,2.155534e-04,1683773760,0.008999,0.008999,"{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9188.86', 'barrier_count': 1, 'bid_price': 0.00899908, 'buy_price': '0.01000000', 'contract_id': '7839', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9188.86', 'barrier_count': 1, 'bid_price': 0.00994033, 'buy_price': '0.01000000', 'contract_id': '7839', 'contract_type': 'VANILLA...",1683777527,0.008753,0.008797,"{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683772930, 'tick': '9144.1', 'tick_display_value': '9144.10'}, {'epoch': 1683772931, 'tick': '..."
8,1HZ10V,VANILLALONGPUT,9144.8800,1683772938,0.010000,22h,+24.20,9.169080e+03,"[1683772938, 1683852138]",7859,...,3.003451e-04,1683773762,0.008922,0.008922,"{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9169.08', 'barrier_count': 1, 'bid_price': 0.00892173, 'buy_price': '0.01000000', 'contract_id': '7859', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9169.08', 'barrier_count': 1, 'bid_price': 0.00976079, 'buy_price': '0.01000000', 'contract_id': '7859', 'contract_type': 'VANILLA...",1683777532,0.008665,0.008687,"{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683772936, 'tick': '9144.61', 'tick_display_value': '9144.61'}, {'epoch': 1683772937, 'tick': ..."
19,R_25,VANILLALONGPUT,1914.4210,1683773192,0.010000,227d,1600.000,1.600000e+03,"[1683773193, 1703462399]",8459,...,2.747835e-04,1683773788,0.008746,0.008746,"{'echo_req': {'contract_id': 8459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '1600.000', 'barrier_count': 1, 'bid_price': 0.00874599, 'buy_price': '0.01000000', 'contract_id': '8459', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 8459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '1600.000', 'barrier_count': 1, 'bid_price': 0.00875182, 'buy_price': '0.01000000', 'contract_id': '8459', 'contract_type': 'VANILL...",1683777586,0.009022,0.009028,"{'echo_req': {'contract_id': 8459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683773188, 'tick': '1914.606', 'tick_display_value': '1914.606'}, {'epoch': 1683773190, 'tick'..."
36,1HZ25V,VANILLALONGPUT,446433.5800,1683773487,0.010000,1299m,-7013.60,4.394200e+05,"[1683773487, 1683851427]",8819,...,3.452900e-05,1683773831,0.007269,0.007269,"{'echo_req': {'contract_id': 8819, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '439419.98', 'barrier_count': 1, 'bid_price': 0.00726886, 'buy_price': '0.01000000', 'contract_id': '8819', 'contract_type': 'VANIL...","{'echo_req': {'contract_id': 8819, 'proposal_open_contract': 1}, 'msg_type

In [1267]:
calc_sold = [] 
sold_time = []
api_sold = []
sold_prop = []
for i in df2.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)               
        proposal = (df2.loc[i].Sold_prop)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, -1) )        
        sold_prop.append(proposal)    
df2['Sold_time'] = sold_time
df2['API_sold'] = api_sold
df2['Calc_sold'] = calc_sold
df2['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(df2), 'Test cases','\nMismatches =',\
      len(df2[round(df2.API_sold,8) - np.round(df2.Calc_sold,8) != 0]),\
      '\nMismatches listed below :')
df22 = df2[round(df2.API_sold,8) - np.round(df2.Calc_sold,8) != 0]
df22

13 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,...,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json,Sold_time,API_sold,Calc_sold,Sold_prop


In [1275]:
calc_sold = [] 
sold_time = []
api_sold = []
sold_prop = []
for i in Sold_df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)              
        proposal = Sold_df.loc[i].Sold_prop
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )        
        sold_prop.append(proposal)    
Sold_df['Sold_time'] = sold_time
Sold_df['API_sold'] = api_sold
Sold_df['Calc_sold'] = calc_sold
Sold_df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[round(Sold_df.API_sold,8) - round(Sold_df.Calc_sold,8) != 0]),\
      '\nMismatches listed below :')
df = Sold_df[round(Sold_df.API_sold,8) - round(Sold_df.Calc_sold,8) != 0]
df

90 Test cases 
Mismatches = 26 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,...,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json,Sold_time,API_sold,Calc_sold,Sold_prop
3,R_100,VANILLALONGCALL,4923.3700,1683772910,0.010000,912m,+273.20,5.196570e+03,"[1683772911, 1683827631]",7759,...,9.444240e-04,1683773750,0.007051,0.007052,"{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00705128, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '5196.57', 'barrier_count': 1, 'bid_price': 0.00828796, 'buy_price': '0.01000000', 'contract_id': '7759', 'contract_type': 'VANILLA...",1683777506,0.004771,0.004771,"{'echo_req': {'contract_id': 7759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683772906, 'tick': '4921.97', 'tick_display_value': '4921.97'}, {'epoch': 1683772908, 'tick': ..."
5,R_10,VANILLALONGCALL,6690.7690,1683772920,0.010000,22h,+43.190,6.733959e+03,"[1683772921, 1683852121]",7799,...,5.765034e-03,1683773754,0.010436,0.010437,"{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '6733.959', 'barrier_count': 1, 'bid_price': 0.01043643, 'buy_price': '0.01000000', 'contract_id': '7799', 'contract_type': 'VANILL...","{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '6733.959', 'barrier_count': 1, 'bid_price': 0.00838459, 'buy_price': '0.01000000', 'contract_id': '7799', 'contract_type': 'VANILL...",1683777516,0.008699,0.008699,"{'echo_req': {'contract_id': 7799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683772916, 'tick': '6690.969', 'tick_display_value': '6690.969'}, {'epoch': 1683772918, 'tick'..."
7,1HZ10V,VANILLALONGPUT,9144.2600,1683772932,0.010000,755m,+44.60,9.188860e+03,"[1683772932, 1683818232]",7839,...,2.155534e-04,1683773760,0.008999,0.008999,"{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9188.86', 'barrier_count': 1, 'bid_price': 0.00899908, 'buy_price': '0.01000000', 'contract_id': '7839', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9188.86', 'barrier_count': 1, 'bid_price': 0.00994033, 'buy_price': '0.01000000', 'contract_id': '7839', 'contract_type': 'VANILLA...",1683777527,0.008753,0.008797,"{'echo_req': {'contract_id': 7839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'audit_details': {'contract_start': [{'epoch': 1683772930, 'tick': '9144.1', 'tick_display_value': '9144.10'}, {'epoch': 1683772931, 'tick': '..."
8,1HZ10V,VANILLALONGPUT,9144.8800,1683772938,0.010000,22h,+24.20,9.169080e+03,"[1683772938, 1683852138]",7859,...,3.003451e-04,1683773762,0.008922,0.008922,"{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200201', 'barrier': '9169.08', 'barrier_count': 1, 'bid_price': 0.00892173, 'buy_price': '0.01000000', 'contract_id': '7859', 'contract_type': 'VANILLA...","{'echo_req': {'contract_id': 7859, 'proposal_open_contract': 1}, 'msg_type': 'pro

In [598]:
Expired_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
                               'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'API_exp', 'Calc_exp','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m')
while True:    
    print(len(Expired_df))    
    if len(Expired_df)> 30-1:
        break            
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365        
    duration = np.random.randint(1, 5)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Expired_df.loc[len(Expired_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, proposal]                               
    clear_output(wait=True)
Expired_df

,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time
0,R_10,VANILLALONGPUT,9977.4140,1683537136,0.001,4m,+1.440,9978.8540,"[1683537136, 1683537376]",4199,0,0.000495,0.000979,0.000979,"{'echo_req': {'contract_id': 4199, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'barrier': '9978.854', 'barrier_count': 1, 'bid_price': 0.00094175, 'buy_price': '0.00100000', 'contract_id': '4199', 'contract_type': 'VANILL...",1683537376
1,R_75,VANILLALONGCALL,9954.6879,1683537140,0.001,2m,+0.0110,9954.6989,"[1683537141, 1683537261]",4219,0,0.000169,0.000000,0.000000,"{'echo_req': {'contract_id': 4219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'barrier': '9954.6989', 'barrier_count': 1, 'bid_price': 0.00130924, 'buy_price': '0.00100000', 'contract_id': '4219', 'contract_type': 'VANIL...",1683537261
2,1HZ100V,VANILLALONGPUT,10010.8000,1683537147,0.001,3m,+12.60,10023.4000,"[1683537147, 1683537327]",4239,0,0.000056,0.000000,0.000000,"{'echo_req': {'contract_id': 4239, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'barrier': '10023.40', 'barrier_count': 1, 'bid_price': 0.00091693, 'buy_price': '0.00100000', 'contract_id': '4239', 'contract_type': 'VANILL...",1683537327
3,1HZ100V,VANILLALONGPUT,10009.2100,1683537152,0.001,4m,+35.50,10044.7100,"[1683537152, 1683537392]",4259,0,0.000027,0.000962,0.000962,"{'echo_req': {'contract_id': 4259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'barrier': '10044.71', 'barrier_count': 1, 'bid_price': 0.00096952, 'buy_price': '0.00100000', 'contract_id': '4259', 'contract_type': 'VANILL...",1683537392
4,1HZ25V,VANILLALONGPUT,9995.6100,1683537158,0.001,1m,+0.00,9995.6100,"[1683537158, 1683537218]",4279,0,0.000709,0.000000,0.000000,"{'echo_req': {'contract_id': 4279, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'barrier': '9995.61', 'barrier_count': 1, 'bid_price': 0.00073962, 'buy_price': '0.00100000', 'contract_id': '4279', 'contract_type': 'VANILLA...",1683537218
5,R_10,VANILLALONGCALL,9977.9160,1683537162,0.001,4m,+3.510,9981.4260,"[1683537163, 1683537403]",4319,0,0.006782,0.000000,0.000000,"{'echo_req': {'contract_id': 4319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'barrier': '9981.426', 'barrier_count': 1, 'bid_price': 0.00084419, 'buy_price': '0.00100000', 'contract_id': '4319', 'contract_type': 'VANILL...",1683537403
6,1HZ75V,VANILLALONGPUT,9808.7200,1683537169,0.001,4m,+0.00,9808.7200,"[1683537169, 1683537409]",4339,0,0.000118,0.002662,0.002662,"{'echo_req': {'contract_id': 4339, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'barrier': '9808.72', 'barrier_count': 1, 'bid_price': 0.00090814, 'buy_price': '0.00100000', 'contract_id': '4339', 'contract_type': 'VANILLA...",1683537409
7,1HZ100V,VANILLALONGCALL,10009.0400,1683537174,0.001,1m,+17.70,10026.7400,"[1683537174, 1683537234]",4379,0,0.000818,0.000000,0.000000,"{'echo_req': {'contract_id': 4379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'barrier': '10026.74', 'barrier_count': 1, 'bid_price': '0.00007416', 'buy_price': '0.00100000', 'contract_id': '4379', 'contract_type': 'VANI...",1683537234
8,1HZ50V,VANILLALONGPUT,9961.3100,1683537185,0.001,2m,-5.10,9956.2100,"[1683537185, 1683537305]",4419,0,0.000373,0.000000,0.000000,"{'echo_req': {'contract_id': 4419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200161', 'ba

In [600]:
calc_exp = [] 
exp_time = []
api_exp = []
for i in Expired_df.index:   
    if i >= len(exp_time):      
        clear_output(wait = True)                
        # getting proposals for expired contracts to save API payout
        proposal = get_proposal(Expired_df.loc[i].ID)        
        if proposal['proposal_open_contract']['is_sold'] == 1:            
            exp_time.append(proposal['proposal_open_contract']['date_expiry'])
            api_exp.append(float(proposal['proposal_open_contract']['sell_price']))
            # Calculating expired payout to compare againts api
            calc_exp.append( calc_ppp_exp(proposal, 0, 0) )            
        else:           
            exp_time.append(0)            
            api_exp.append(0)
            calc_exp.append(0)    
Expired_df['Exp_time'] = exp_time
Expired_df['API_exp'] = api_exp
Expired_df['Calc_exp'] = calc_exp
clear_output(wait = True)
print(len(Expired_df), 'Test cases','\nMismatches =',\
      len(Expired_df[round(Expired_df.API_exp,8) - round(Expired_df.Calc_exp,8) != 0]),\
      '\nMismatches listed below :')
df = Expired_df[round(Expired_df.API_exp,8) - round(Expired_df.Calc_exp,8) != 0]
df

20 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time
